In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.keras.applications import MobileNetV3Small
import time
keras = tf.keras
layers = tf.keras.layers
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0  # normalize to [0,1] range
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\1\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\1\test'
checkpoint_path = r'C:\Users\24306\Desktop\mobilenetv3\1\mobilenetv3.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
#print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(len(train_image_label_ds))



18926


# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
#print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(len(test_image_label_ds))

4738


In [6]:
MobileNetV3 = MobileNetV3Small(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
MobileNetV3.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
x = MobileNetV3(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
MobilenetV3small (Functional (None, 1, 1, 1024)        1529968   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
out_weight (Dense)           (None, 1)                 1025      
Total params: 1,530,993
Trainable params: 1,518,881
Non-trainable params: 12,112
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
np.savetxt('mobilenetv3_loss1.txt',history.history['loss'])
#np.savetxt('DenseNet201_acc.txt',history.history['acc'])
np.savetxt('mobilenetv3_val_loss1.txt',history.history['val_loss'])
#np.savetxt('DenseNet201_val_acc.txt',history.history['val_acc'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Sat Apr  1 09:33:36 2023
Epoch 1/300
591/591 [==============================] - 39s 46ms/step - loss: 48.1122 - val_loss: 2812.1550
INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\mobilenetv3\1\mobilenetv3.ckpt\assets


C:\Users\24306\.conda\envs\keras\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/300
591/591 [==============================] - 28s 48ms/step - loss: 3.4625 - val_loss: 2936.5781
Epoch 3/300
591/591 [==============================] - 29s 49ms/step - loss: 1.9288 - val_loss: 2900.3015
Epoch 4/300
591/591 [==============================] - 28s 48ms/step - loss: 1.4012 - val_loss: 2296.6138
INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\mobilenetv3\1\mobilenetv3.ckpt\assets


C:\Users\24306\.conda\envs\keras\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 5/300
591/591 [==============================] - 31s 52ms/step - loss: 1.1355 - val_loss: 1967.8051
INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\mobilenetv3\1\mobilenetv3.ckpt\assets


...

Epoch 235/300
591/591 [==============================] - 60s 101ms/step - loss: 0.1727 - val_loss: 0.1366
Epoch 236/300
591/591 [==============================] - 59s 100ms/step - loss: 0.1717 - val_loss: 0.1666
Epoch 237/300
591/591 [==============================] - 59s 100ms/step - loss: 0.1778 - val_loss: 0.1622
Epoch 238/300
591/591 [==============================] - 60s 102ms/step - loss: 0.1679 - val_loss: 0.1548
Epoch 239/300
591/591 [==============================] - 60s 102ms/step - loss: 0.1986 - val_loss: 0.1470
Epoch 240/300
591/591 [==============================] - 61s 103ms/step - loss: 0.1681 - val_loss: 0.1376
Epoch 241/300
591/591 [==============================] - 62s 105ms/step - loss: 0.1784 - val_loss: 0.1364
Epoch 242/300
591/591 [==============================] - 59s 100ms/step - loss: 0.1964 - val_loss: 0.1229
Epoch 243/300
591/591 [==============================] - 60s 101ms/step - loss: 0.1639 - val_loss: 0.1193
Epoch 244/300
591/591 [=======================

C:\Users\24306\.conda\envs\keras\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 283/300
591/591 [==============================] - 62s 104ms/step - loss: 0.1635 - val_loss: 0.1158
Epoch 284/300
591/591 [==============================] - 61s 103ms/step - loss: 0.1598 - val_loss: 0.1450
Epoch 285/300
591/591 [==============================] - 61s 103ms/step - loss: 0.1634 - val_loss: 0.1373
Epoch 286/300
591/591 [==============================] - 61s 102ms/step - loss: 0.1747 - val_loss: 0.1538
Epoch 287/300
591/591 [==============================] - 61s 103ms/step - loss: 0.1604 - val_loss: 0.1852
Epoch 288/300
591/591 [==============================] - 61s 103ms/step - loss: 0.1566 - val_loss: 0.1183
Epoch 289/300
591/591 [==============================] - 61s 103ms/step - loss: 0.1591 - val_loss: 0.1086
INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\mobilenetv3\1\mobilenetv3.ckpt\assets


C:\Users\24306\.conda\envs\keras\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 290/300
591/591 [==============================] - 62s 105ms/step - loss: 0.1657 - val_loss: 0.1167
Epoch 291/300
591/591 [==============================] - 62s 105ms/step - loss: 0.1613 - val_loss: 0.1293
Epoch 292/300
591/591 [==============================] - 62s 105ms/step - loss: 0.1660 - val_loss: 0.1578
Epoch 293/300
591/591 [==============================] - 62s 104ms/step - loss: 0.1788 - val_loss: 0.1259
Epoch 294/300
591/591 [==============================] - 62s 104ms/step - loss: 0.1633 - val_loss: 0.1367
Epoch 295/300
591/591 [==============================] - 62s 104ms/step - loss: 0.1872 - val_loss: 0.1683
Epoch 296/300
591/591 [==============================] - 62s 104ms/step - loss: 0.1640 - val_loss: 0.1403
Epoch 297/300
591/591 [==============================] - 62s 105ms/step - loss: 0.1689 - val_loss: 0.1451
Epoch 298/300
591/591 [==============================] - 61s 104ms/step - loss: 0.1826 - val_loss: 0.1363
Epoch 299/300
591/591 [=======================